In [22]:
import pandas as pd
from pathlib import Path

# base paths
raw_dir = Path("../raw/10k_synthea_covid19_csv")
interim_dir = Path("../interim")
interim_dir.mkdir(parents=True, exist_ok=True)

raw_dir, raw_dir.exists()


(WindowsPath('../raw/10k_synthea_covid19_csv'), True)

In [23]:
patients_path = raw_dir / "patients.csv"
print("patients.csv exists:", patients_path.exists())

df_pat = pd.read_csv(patients_path)
df_pat.head()


patients.csv exists: True


,Id,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,LAST,SUFFIX,...,BIRTHPLACE,ADDRESS,CITY,STATE,COUNTY,ZIP,LAT,LON,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE
0,f0f3bc8d-ef38-49ce-a2bd-dfdda982b271,2017-08-24,NaN,999-68-6630,NaN,NaN,NaN,Jacinto644,Kris249,NaN,...,Beverly Massachusetts US,888 Hickle Ferry Suite 38,Springfield,Massachusetts,Hampden County,1106.0,42.151961,-72.598959,8446.49,1499.08
1,067318a4-db8f-447f-8b6e-f2f61e9baaa5,2016-08-01,NaN,999-15-5895,NaN,NaN,NaN,Alva958,Krajcik437,NaN,...,Boston Massachusetts US,1048 Skiles Trailer,Walpole,Massachusetts,Norfolk County,2081.0,42.177370,-71.281353,89893.40,1845.72
2,ae9efba3-ddc4-43f9-a781-f72019388548,1992-06-30,NaN,999-27-3385,S99971451,X53218815X,Mr.,Jayson808,Fadel536,NaN,...,Springfield Massachusetts US,1056 Harris Lane Suite 70,Chicopee,Massachusetts,Hampden County,1020.0,42.181642,-72.608842,577445.86,3528.84
3,199c586f-af16-4091-9998-ee4cfc02ee7a,2004-01-09,NaN,999-73-2461,S99956432,NaN,NaN,Jimmie93,Harris789,NaN,...,Worcester Massachusetts US,201 Mitchell Lodge Unit 67,Pembroke,Massachusetts,Plymouth County,NaN,42.075292,-70.757035,336701.72,2705.64
4,353016ea-a0ff-4154-85bb-1cf8b6cedf20,1996-11-15,NaN,999-60-7372,S99917327,X58903159X,Mr.,Gregorio366,Auer97,NaN,...,Patras Achaea GR,1050 Lindgren Extension Apt 38,Boston,Massachusetts,Suffolk County,2135.0,42.352434,-71.028610,484076.34,3043.04


In [24]:
def build_simple_note(row):
    first = str(row.get("FIRST", "")).title()
    last = str(row.get("LAST", "")).title()
    birthdate = str(row.get("BIRTHDATE", ""))
    address = str(row.get("ADDRESS", ""))
    city = str(row.get("CITY", ""))
    state = str(row.get("STATE", ""))
    zip_code = str(row.get("ZIP", ""))
    ssn = str(row.get("SSN", ""))
    drivers = str(row.get("DRIVERS", ""))
    passport = str(row.get("PASSPORT", ""))

    parts = []

    if first or last:
        parts.append(f"Patient {first} {last}.")

    if birthdate and birthdate != "nan":
        parts.append(f"They were born on {birthdate}.")

    if address or city or state or zip_code:
        parts.append(f"They live at {address}, {city}, {state} {zip_code}.")

    if ssn and ssn != "nan":
        parts.append(f"Their social security number is {ssn}.")

    if drivers and drivers != "nan":
        parts.append(f"Their driver's license number is {drivers}.")

    if passport and passport != "nan":
        parts.append(f"Their passport number is {passport}.")

    return " ".join(p for p in parts if p)

# base patient notes dataframe (will be our anchor for merging)
df_notes = df_pat.copy()
df_notes["note"] = df_notes.apply(build_simple_note, axis=1)

df_notes[["note"]].head()


,note
0,Patient Jacinto644 Kris249. They were born on ...
1,Patient Alva958 Krajcik437. They were born on ...
2,Patient Jayson808 Fadel536. They were born on ...
3,Patient Jimmie93 Harris789. They were born on ...
4,Patient Gregorio366 Auer97. They were born on ...


In [25]:
encounters_path = raw_dir / "encounters.csv"
conditions_path = raw_dir / "conditions.csv"
medications_path = raw_dir / "medications.csv"
providers_path = raw_dir / "providers.csv"

# payer file can be "payers.csv" or "payer.csv" depending on the bundle
payer_path = raw_dir / "payers.csv"
if not payer_path.exists():
    payer_path = raw_dir / "payer.csv"

print("encounters.csv:", encounters_path.exists())
print("conditions.csv:", conditions_path.exists())
print("medications.csv:", medications_path.exists())
print("providers.csv:", providers_path.exists())
print("payer(s).csv:", payer_path.exists())

df_enc = pd.read_csv(encounters_path)
df_cond = pd.read_csv(conditions_path)
df_med = pd.read_csv(medications_path)
df_prov = pd.read_csv(providers_path)
df_payer = pd.read_csv(payer_path)

df_enc.head()


encounters.csv: True
conditions.csv: True
medications.csv: True
providers.csv: True
payer(s).csv: True


,Id,START,STOP,PATIENT,ORGANIZATION,PROVIDER,PAYER,ENCOUNTERCLASS,CODE,DESCRIPTION,BASE_ENCOUNTER_COST,TOTAL_CLAIM_COST,PAYER_COVERAGE,REASONCODE,REASONDESCRIPTION
0,d5ee30a9-362f-429e-a87a-ee38d999b0a5,2019-02-16T01:02:32Z,2019-02-16T01:17:32Z,f0f3bc8d-ef38-49ce-a2bd-dfdda982b271,5103c940-0c08-392f-95cd-446e0cea042a,e2c226c2-3e1e-3d0b-b997-ce9544c10528,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,outpatient,185345009,Encounter for symptom,129.16,129.16,69.16,65363002.0,Otitis media
1,6a74fdef-2287-44bf-b9e7-18012376faca,2019-08-02T01:02:32Z,2019-08-02T01:32:32Z,f0f3bc8d-ef38-49ce-a2bd-dfdda982b271,0b9f3f7c-8ab6-30a5-b3ae-4dc0e0c00cb3,87c33fc5-3fd1-3c52-815a-b89a1623bb3a,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,wellness,410620009,Well child visit (procedure),129.16,129.16,129.16,NaN,NaN
2,8bca6d8a-ab80-4cbf-8abb-46654235f227,2019-10-31T01:02:32Z,2019-10-31T01:17:32Z,f0f3bc8d-ef38-49ce-a2bd-dfdda982b271,5103c940-0c08-392f-95cd-446e0cea042a,e2c226c2-3e1e-3d0b-b997-ce9544c10528,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,outpatient,185345009,Encounter for symptom,129.16,129.16,69.16,65363002.0,Otitis media
3,821e57ac-9304-46a9-9f9b-83daf60e9e43,2020-01-31T01:02:32Z,2020-01-31T01:17:32Z,f0f3bc8d-ef38-49ce-a2bd-dfdda982b271,0b9f3f7c-8ab6-30a5-b3ae-4dc0e0c00cb3,87c33fc5-3fd1-3c52-815a-b89a1623bb3a,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,wellness,410620009,Well child visit (procedure),129.16,129.16,129.16,NaN,NaN
4,681c380b-3c84-4c55-80a6-db3d9ea12fee,2020-03-02T01:02:32Z,2020-03-02T01:58:32Z,f0f3bc8d-ef38-49ce-a2bd-dfdda982b271,fd328395-ab1d-35c6-a2d0-d05a9a79cf11,9c875a09-93e0-39aa-9260-ad264bbdd3fe,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,ambulatory,185345009,Encounter for symptom (procedure),129.16,129.16,69.16,NaN,NaN


In [26]:
# patients: Id -> PATIENT_ID
df_pat_ids = df_pat.rename(columns={"Id": "PATIENT_ID"})

# conditions: group a few per patient
cond_text_col = "DESCRIPTION" if "DESCRIPTION" in df_cond.columns else "CODE"
df_cond_grouped = (
    df_cond.groupby("PATIENT")[cond_text_col]
    .apply(lambda s: ", ".join(s.astype(str).unique()[:3]))
    .reset_index()
    .rename(columns={cond_text_col: "COND_SUMMARY", "PATIENT": "PATIENT_ID"})
)

# medications: group a few per patient
med_text_col = "DESCRIPTION" if "DESCRIPTION" in df_med.columns else "CODE"
df_med_grouped = (
    df_med.groupby("PATIENT")[med_text_col]
    .apply(lambda s: ", ".join(s.astype(str).unique()[:3]))
    .reset_index()
    .rename(columns={med_text_col: "MED_SUMMARY", "PATIENT": "PATIENT_ID"})
)

df_cond_grouped.head(), df_med_grouped.head()


(                             PATIENT_ID  \
 0  0000b247-1def-417a-a783-41c8682be022   
 1  00049ee8-5953-4edd-a277-b9c1b1a7f16b   
 2  000769a6-23a7-426e-a264-cb0e509b2da2   
 3  00079a57-24a8-430f-b4f8-a1cf34f90060   
 4  0008a63c-c95c-46c2-9ef3-831d68892019   
 
                                         COND_SUMMARY  
 0  Headache (finding), Cough (finding), Sputum fi...  
 1  Perennial allergic rhinitis with seasonal vari...  
 2  Chronic intractable migraine without aura, Ost...  
 3  Hypertension, Cough (finding), Muscle pain (fi...  
 4  Chronic sinusitis (disorder), Body mass index ...  ,
                              PATIENT_ID  \
 0  00049ee8-5953-4edd-a277-b9c1b1a7f16b   
 1  000769a6-23a7-426e-a264-cb0e509b2da2   
 2  00079a57-24a8-430f-b4f8-a1cf34f90060   
 3  0008a63c-c95c-46c2-9ef3-831d68892019   
 4  00093cdd-a9f0-4ad8-87e9-53534501f008   
 
                                          MED_SUMMARY  
 0  Chlorpheniramine Maleate 2 MG/ML Oral Solution...  
 1  Naproxen sodium

In [27]:
# providers: Id -> PROVIDER_ID
df_prov_simple = df_prov.rename(columns={"Id": "PROVIDER_ID"}).copy()

# try to build provider full name
if "NAME" in df_prov_simple.columns:
    df_prov_simple["PROVIDER_NAME"] = df_prov_simple["NAME"]
elif {"FIRST", "LAST"}.issubset(df_prov_simple.columns):
    df_prov_simple["PROVIDER_NAME"] = (
        df_prov_simple["FIRST"].astype(str) + " " + df_prov_simple["LAST"].astype(str)
    )
else:
    df_prov_simple["PROVIDER_NAME"] = df_prov_simple["PROVIDER_ID"].astype(str)

# join into encounters
df_enc_prov = df_enc.merge(
    df_prov_simple[["PROVIDER_ID", "PROVIDER_NAME"]],
    left_on="PROVIDER",
    right_on="PROVIDER_ID",
    how="left",
)

# aggregate encounters: latest date + provider names per patient
enc_date_col = "START" if "START" in df_enc_prov.columns else df_enc_prov.columns[0]

df_enc_grouped = (
    df_enc_prov.groupby("PATIENT")
    .agg({
        enc_date_col: lambda s: s.astype(str).max(),
        "PROVIDER_NAME": lambda s: ", ".join(s.dropna().astype(str).unique()[:2]),
    })
    .reset_index()
    .rename(columns={
        "PATIENT": "PATIENT_ID",
        enc_date_col: "LAST_ENC_DATE"
    })
)

df_enc_grouped.head()


,PATIENT_ID,LAST_ENC_DATE,PROVIDER_NAME
0,0000b247-1def-417a-a783-41c8682be022,2020-02-18T14:47:48Z,"Leontine92 Lebsack687, Denver542 Mills423"
1,00049ee8-5953-4edd-a277-b9c1b1a7f16b,2020-03-08T15:49:20Z,"Efrain317 Hauck852, Anderson154 Schoen8"
2,000769a6-23a7-426e-a264-cb0e509b2da2,2020-04-10T01:41:27Z,"Kevin729 Kovacek682, Anibal473 Sawayn19"
3,00079a57-24a8-430f-b4f8-a1cf34f90060,2020-03-08T13:37:02Z,"Ramon749 Rath779, Wilber603 Hammes673"
4,0008a63c-c95c-46c2-9ef3-831d68892019,2020-03-18T21:03:20Z,"Lyndsey828 Schultz619, Ana972 Serna195"


In [28]:
# payers: Id -> PAYER_ID, assume NAME column
df_payer_ids = df_payer.rename(columns={"Id": "PAYER_ID"})

if "PAYER" in df_enc.columns:
    df_enc_payer = df_enc[["PATIENT", "PAYER"]].dropna()
    df_enc_payer = df_enc_payer.merge(
        df_payer_ids[["PAYER_ID", "NAME"]].rename(columns={"NAME": "PAYER_NAME"}),
        left_on="PAYER",
        right_on="PAYER_ID",
        how="left"
    )
    df_payer_grouped = (
        df_enc_payer.groupby("PATIENT")["PAYER_NAME"]
        .apply(lambda s: ", ".join(s.dropna().astype(str).unique()[:2]))
        .reset_index()
        .rename(columns={"PATIENT": "PATIENT_ID", "PAYER_NAME": "PAYER_SUMMARY"})
    )
else:
    df_payer_grouped = pd.DataFrame(columns=["PATIENT_ID", "PAYER_SUMMARY"])

df_payer_grouped.head()


,PATIENT_ID,PAYER_SUMMARY
0,0000b247-1def-417a-a783-41c8682be022,Humana
1,00049ee8-5953-4edd-a277-b9c1b1a7f16b,Blue Cross Blue Shield
2,000769a6-23a7-426e-a264-cb0e509b2da2,"Aetna, Medicare"
3,00079a57-24a8-430f-b4f8-a1cf34f90060,"Humana, NO_INSURANCE"
4,0008a63c-c95c-46c2-9ef3-831d68892019,Medicaid


In [29]:
df_rich = df_pat_ids.copy()

for extra in [df_cond_grouped, df_med_grouped, df_enc_grouped, df_payer_grouped]:
    if not extra.empty:
        df_rich = df_rich.merge(extra, on="PATIENT_ID", how="left")

def build_rich_note(row):
    first = str(row.get("FIRST", "")).title()
    last = str(row.get("LAST", "")).title()
    birthdate = str(row.get("BIRTHDATE", ""))
    address = str(row.get("ADDRESS", ""))
    city = str(row.get("CITY", ""))
    state = str(row.get("STATE", ""))
    zip_code = str(row.get("ZIP", ""))
    ssn = str(row.get("SSN", ""))
    drivers = str(row.get("DRIVERS", ""))
    passport = str(row.get("PASSPORT", ""))

    cond_summary = str(row.get("COND_SUMMARY", ""))
    med_summary = str(row.get("MED_SUMMARY", ""))
    enc_date = str(row.get("LAST_ENC_DATE", ""))
    provider_name = str(row.get("PROVIDER_NAME", ""))
    payer_summary = str(row.get("PAYER_SUMMARY", ""))

    parts = []

    if first or last:
        parts.append(f"Patient {first} {last}.")

    if birthdate and birthdate != "nan":
        parts.append(f"They were born on {birthdate}.")

    if address or city or state or zip_code:
        parts.append(f"They live at {address}, {city}, {state} {zip_code}.")

    if ssn and ssn != "nan":
        parts.append(f"Their social security number is {ssn}.")

    if drivers and drivers != "nan":
        parts.append(f"Their driver's license number is {drivers}.")

    if passport and passport != "nan":
        parts.append(f"Their passport number is {passport}.")

    if cond_summary and cond_summary != "nan":
        parts.append(f"They have been diagnosed with {cond_summary}.")

    if med_summary and med_summary != "nan":
        parts.append(f"Their current medications include {med_summary}.")

    if enc_date and enc_date != "nan":
        parts.append(f"Their last recorded encounter was on {enc_date}.")

    if provider_name and provider_name != "nan":
        parts.append(f"Their primary provider is {provider_name}.")

    if payer_summary and payer_summary != "nan":
        parts.append(f"Their insurance coverage is through {payer_summary}.")

    return " ".join(p for p in parts if p)

df_rich["note_rich"] = df_rich.apply(build_rich_note, axis=1)
df_rich[["note_rich"]].head()


,note_rich
0,Patient Jacinto644 Kris249. They were born on ...
1,Patient Alva958 Krajcik437. They were born on ...
2,Patient Jayson808 Fadel536. They were born on ...
3,Patient Jimmie93 Harris789. They were born on ...
4,Patient Gregorio366 Auer97. They were born on ...


In [30]:
staff_data = [
    {"staff_id": "S001", "name": "Dr. Meera Rao", "role": "Cardiologist",
     "email": "meera.rao@hospital.org", "phone": "555-1001", "department": "Cardiology"},
    {"staff_id": "S002", "name": "Dr. Arjun Patel", "role": "Oncologist",
     "email": "arjun.patel@hospital.org", "phone": "555-1002", "department": "Oncology"},
    {"staff_id": "S003", "name": "Nurse Priya Singh", "role": "Head Nurse",
     "email": "priya.singh@hospital.org", "phone": "555-2001", "department": "ICU"},
    {"staff_id": "S004", "name": "Dr. Rohit Menon", "role": "Surgeon",
     "email": "rohit.menon@hospital.org", "phone": "555-3001", "department": "Surgery"},
    {"staff_id": "S005", "name": "Admin Riya Das", "role": "Administrator",
     "email": "riya.das@hospital.org", "phone": "555-4001", "department": "Billing"},
]

df_staff = pd.DataFrame(staff_data)

def build_staff_note(row):
    return (
        f"Staff member {row['name']} with staff ID {row['staff_id']} works as a "
        f"{row['role']} in the {row['department']} department. "
        f"Their contact email is {row['email']} and phone number is {row['phone']}."
    )

df_staff["note_staff"] = df_staff.apply(build_staff_note, axis=1)
df_staff[["note_staff"]].head()


,note_staff
0,Staff member Dr. Meera Rao with staff ID S001 ...
1,Staff member Dr. Arjun Patel with staff ID S00...
2,Staff member Nurse Priya Singh with staff ID S...
3,Staff member Dr. Rohit Menon with staff ID S00...
4,Staff member Admin Riya Das with staff ID S005...


In [31]:
finance_data = [
    {"invoice_id": "I1001", "patient_id": "PATIENT-1", "amount": 450.0,
     "payer_name": "BlueCross Gold", "account_number": "ACCT-778833", "status": "PAID"},
    {"invoice_id": "I1002", "patient_id": "PATIENT-2", "amount": 1200.0,
     "payer_name": "MediCare Plus", "account_number": "ACCT-991122", "status": "PENDING"},
    {"invoice_id": "I1003", "patient_id": "PATIENT-3", "amount": 800.0,
     "payer_name": "City Health Plan", "account_number": "ACCT-551199", "status": "OVERDUE"},
    {"invoice_id": "I1004", "patient_id": "PATIENT-4", "amount": 2200.0,
     "payer_name": "BlueCross Silver", "account_number": "ACCT-663344", "status": "PAID"},
    {"invoice_id": "I1005", "patient_id": "PATIENT-5", "amount": 300.0,
     "payer_name": "MediCare Basic", "account_number": "ACCT-110022", "status": "PENDING"},
]

df_fin = pd.DataFrame(finance_data)

def build_fin_note(row):
    return (
        f"Invoice {row['invoice_id']} for patient {row['patient_id']} has an amount of "
        f"${row['amount']:.2f}. The payer is {row['payer_name']} with account number "
        f"{row['account_number']}. The current status of this invoice is {row['status']}."
    )

df_fin["note_fin"] = df_fin.apply(build_fin_note, axis=1)
df_fin[["note_fin"]].head()


,note_fin
0,Invoice I1001 for patient PATIENT-1 has an amo...
1,Invoice I1002 for patient PATIENT-2 has an amo...
2,Invoice I1003 for patient PATIENT-3 has an amo...
3,Invoice I1004 for patient PATIENT-4 has an amo...
4,Invoice I1005 for patient PATIENT-5 has an amo...


In [32]:
extra_notes = []

# rich patient notes
for text in df_rich["note_rich"].astype(str).tolist():
    if text.strip():
        extra_notes.append({"note": text})

# staff notes
for text in df_staff["note_staff"].astype(str).tolist():
    if text.strip():
        extra_notes.append({"note": text})

# finance notes
for text in df_fin["note_fin"].astype(str).tolist():
    if text.strip():
        extra_notes.append({"note": text})

len(extra_notes)


12362

In [33]:
# df_notes has all patient columns + 'note'
df_multi = pd.DataFrame(extra_notes)

# Make sure df_multi has the same columns as df_notes
df_multi = df_multi.reindex(columns=df_notes.columns)

# Concatenate original patient notes + extra multi-source notes
df_combined = pd.concat([df_notes, df_multi], ignore_index=True)

df_combined[["note"]].head(), df_combined.shape


(                                                note
 0  Patient Jacinto644 Kris249. They were born on ...
 1  Patient Alva958 Krajcik437. They were born on ...
 2  Patient Jayson808 Fadel536. They were born on ...
 3  Patient Jimmie93 Harris789. They were born on ...
 4  Patient Gregorio366 Auer97. They were born on ...,
 (24714, 26))

In [34]:
output_path = interim_dir / "patient_notes.csv"
df_combined.to_csv(output_path, index=False)
output_path


WindowsPath('../interim/patient_notes.csv')